**Task 08: Completing missing data**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00


In [54]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g1.parse(github_storage+"/rdf/data01.rdf", format="xml")
g2.parse(github_storage+"/rdf/data02.rdf", format="xml")

<Graph identifier=Nc30af55e55d84cfd924601369fd3aef6 (<class 'rdflib.graph.Graph'>)>

Tarea: lista todos los elementos de la clase Person en el primer grafo (data01.rdf) y completa los campos (given name, family name y email) que puedan faltar con los datos del segundo grafo (data02.rdf). Puedes usar consultas SPARQL o iterar el grafo, o ambas cosas.

In [55]:
for s, p, o in g1:
  print(s, p, o)
print("\n\n")
for s, p, o in g2:
  print(s, p, o)

http://data.org#JohnDoe http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#Given Harry
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/vcard-rdf/3.0#Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Family Doe
http://data.org#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#EMAIL j.smith@da

In [56]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import RDF
g1.namespace_manager.bind('dt', Namespace('http://data.org#'), override=False)
g1.namespace_manager.bind('vcard', Namespace('http://www.w3.org/2001/vcard-rdf/3.0#'), override=False)
g2.namespace_manager.bind('dt', Namespace('http://data.org#'), override=False)
g2.namespace_manager.bind('vcard', Namespace('http://www.w3.org/2001/vcard-rdf/3.0#'), override=False)
dt = Namespace('http://data.org#')
vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

q1 = prepareQuery('''
  SELECT ?person WHERE {
    ?person rdf:type dt:Person
  }
  ''',
  initNs = {"rdf": RDF, "dt": dt}
)

given = prepareQuery('''
  SELECT ?person WHERE {
    ?person rdf:type dt:Person
    FILTER NOT EXISTS {?person vcard:Given ?name}
  }
  ''',
  initNs = {"vcard": vcard, "rdf": RDF, "dt": dt}
)

family_name = prepareQuery('''
  SELECT ?person WHERE {
    ?person rdf:type dt:Person
    FILTER NOT EXISTS {?person vcard:Family ?family}
  }
  ''',
  initNs = {"vcard": vcard, "rdf": RDF, "dt": dt}
)

email = prepareQuery('''
  SELECT ?person WHERE {
    ?person rdf:type dt:Person
    FILTER NOT EXISTS {?person vcard:EMAIL ?email}
  }
  ''',
  initNs = {"vcard": vcard, "rdf": RDF, "dt": dt}
)

for r in g1.query(q1):
  print(r)
print("\n\n")

lista_sujetos_given = []
for r in g1.query(given):
  lista_sujetos_given.append(r[0])
print(lista_sujetos_given)
print("\n\n")

lista_sujetos_family = []
for r in g1.query(family_name):
  lista_sujetos_family.append(r[0])
print(lista_sujetos_family)
print("\n\n")

lista_sujetos_email = []
for r in g1.query(email):
  lista_sujetos_email.append(r[0])
print(lista_sujetos_email)



(rdflib.term.URIRef('http://data.org#SaraJones'),)
(rdflib.term.URIRef('http://data.org#JohnSmith'),)
(rdflib.term.URIRef('http://data.org#JohnDoe'),)
(rdflib.term.URIRef('http://data.org#HarryPotter'),)



[rdflib.term.URIRef('http://data.org#SaraJones'), rdflib.term.URIRef('http://data.org#JohnDoe')]



[rdflib.term.URIRef('http://data.org#SaraJones'), rdflib.term.URIRef('http://data.org#JohnSmith')]



[rdflib.term.URIRef('http://data.org#SaraJones'), rdflib.term.URIRef('http://data.org#HarryPotter')]


In [57]:
for given in lista_sujetos_given:
  for s, p, o in g2.triples((URIRef(given), vcard.Given, None)):
    g1.add((s, p, o))

for s, p, o in g1:
  print(s, p, o)

http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://data.org#JohnDoe http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#Given Harry
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/vcard-rdf/3.0#Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Family Doe
http://data.org#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://

In [58]:
for family in lista_sujetos_family:
  for s, p, o in g2.triples((URIRef(family), vcard.Family, None)):
    g1.add((s, p, o))

for s, p, o in g1:
  print(s, p, o)

http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://data.org#JohnDoe http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#Given Harry
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/vcard-rdf/3.0#Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Family Smith
http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Family Doe
http://data.org#Person http://www.w3.org/1999

In [59]:
for email in lista_sujetos_email:
  for s, p, o in g2.triples((URIRef(email), vcard.EMAIL, None)):
    g1.add((s, p, o))

for s, p, o in g1:
  print(s, p, o)

http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://data.org#JohnDoe http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#Given Harry
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://www.w3.org/2001/vcard-rdf/3.0#EMAIL http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/vcard-rdf/3.0#Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.org#HarryPotter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.org#Person
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Family Smith
http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Family Doe
http://data.org#Person http://www.w3.org/1999